In [57]:
from Crypto.Util.number import inverse,GCD,isPrime,getPrime
from gmpy2 import gcdext,iroot,isqrt
from random import randint

# get n,e,c in all frames
dct = {}
for i in range(21):
    f = open("./frame/Frame{}".format(i),'r')
    s = f.read()
    temp = [int(s[i*256:(i+1)*256],16) for i in range(3)]
    dct["frame{}".format(i)] = temp

In [58]:
# 使用f1的p作为例子
p = bin(7273268163465293471933643674908027120929096536045429682300347130226398442391418956862476173798834057392247872274441320512158525416407044516675402521694747)[2:]
print(len(p))
block_size = 16
chunk = [p[i:i+block_size] for i in range(0,len(p),block_size)]
print(chunk)
print(len(chunk))

512
['1000101011011110', '1111111010000101', '1110001110100000', '1000101100011111', '0101101100110010', '0000011001001001', '1111011000010100', '1101101010000011', '1000110011000110', '1011011001001101', '1110101111001000', '0010110000100111', '1111001110011010', '0101001010010001', '1011100010111100', '0110010000001011', '1010001110101110', '0101111100010101', '1001000011110000', '1010011000101111', '1111000100000010', '1001111111011001', '1110100001100100', '0101011010010011', '0110111110010110', '0001100011011101', '0111001100011000', '0001100100110111', '1111001101101010', '0000111000100001', '0010010100001100', '1101001000011011']
32


In [59]:
# calculate the a,b
x = int(chunk[0],2)
y = int(chunk[1],2)
z = int(chunk[2],2)
m = pow(2,block_size)
a = (z-y)*inverse(y-x,m)%m
b = (y-a*x)%m
print("a = {},b = {}".format(a,b-m))

a = 365,b = -1


In [60]:
# verify the a,b
x = int(chunk[0],2)
print(x,"init",end="\t")
a = 365
b = -1
m = pow(2,block_size)
for i in range(1,len(chunk)):
    y = (a*x+b)%m
    x = y
    if y == int(chunk[i],2):
        print(y,"True",end="\t")
    else:
        print("false",end="\t")

35550 init	65157 True	58272 True	35615 True	23346 True	1609 True	62996 True	55939 True	36038 True	46669 True	60360 True	11303 True	62362 True	21137 True	47292 True	25611 True	41902 True	24341 True	37104 True	42543 True	61698 True	40921 True	59492 True	22163 True	28566 True	6365 True	29464 True	6455 True	62314 True	3617 True	9484 True	53787 True	

In [70]:
def crack_PRG(n):
    a = 365
    b = -1
    m = pow(2,16)
    # j: RandSeed
    for j in range(m):
        x = j
        p = bin(x)[2:].zfill(16)
        for _ in range(63):
            y = (a*x+b)%m
            x = y
            p += bin(x)[2:].zfill(16)
            if n%int(p,2) == 0:
                return int(p,2)
            """
            p的长度不是16的整数倍,所以需要截断处理
            效率极低,跑全部frame时,可将下述代码注释后执行
            跑特定frame时,取消注释
            """
#             if len(p) == 1024:
#                 for plen in range(513,576):
#                     temp = p[:plen]
#                     if n%int(temp,2) == 0:
#                         return int(temp,2)

# for i in [2,13,17]:
for i in range(21):
    f = dct["frame{}".format(i)]
    n = f[0]
    p = crack_PRG(n)
    if p:
        print("{}\n- p: {}\n- q: {}\n".format(i,p,n//p))

2
- p: 52484065122572767557293534477361686456679280880304125291106733197354892893647364164212186415880889674435558369420400890814461263958618375991691022752189839
- q: 1719620105458406433483340568317543019584575635895742560438771105058321655238562613083979651479555788009994557822024565226932906295208262756822275663694111

13
- p: 28159870572597920594563893250499572739237769660647238839011417383170724985058502301163390234256825164330439886062865686161169349465086627567328776299903327
- q: 3351951982485649274893506249551461531869841455148098344430890360930441007518386744200468574541725856922507964546621512713438470702986642490397676148760429

17
- p: 33168227830849222860094691158174263663422336899723339302414624335921937096795361698659264621281924482566876836548283550012466892022559940789243695658550143
- q: 3351951982485649274893506249551461531869841455148098344430890360930441007518386744200468574541725856922511753810518751644117399610974624147782234049900131



### p的长度是16的整数倍
0
- p: 6812427463539231600349464320632373878259506266011361351387035583576132041989251176508636878569037408377298402530520856411137155634830256279629494046731167
- q: 13219767207586640795571526377541732890465157517028423385812256305169857431521906172089205698188779144534548881591815311408600403492889107894773842719465101

1
- p: 7273268163465293471933643674908027120929096536045429682300347130226398442391418956862476173798834057392247872274441320512158525416407044516675402521694747
- q: 12775796067504534889308793837705093856447186276434607181291462366302734214583227473619414509043813033676998357747882057607288385639737162184366176530607467

3
- p: 7055398260479522499340383681532186847092995337600547504968011029334785849248198827261592108738030900350146800969715064939422943632175860165796992047655507
- q: 13078018330994845621552747855429996741129628768867676788710615773048246855802311926334525991481279783903078961691672741494174797757666145632542141818518409

4
- p: 6812427463539231600349464320632373878259506266011361351387035583576132041989251176508636878569037408377298402530520856411137155634830256279629494046731167
- q: 13219767207586640795571526377541732890465157517028423385812256305169857431521906172089205698188779144534548881591815311408600403492889107894773842719465101

5
- p: 8534204848837515931975393694743604482233978795239717717444249645500744498481212186170560396980739879301795276459915005215431744867752453542213016868639743
- q: 11623075998787245402346559750331455079165060117337930034889727146620333293916811257160778523014841986594526205881807563317848049715664071879977965137310113

7
- p: 12406300145307944335209213373018196725715201666535385618794522686524721950743049201561939737652055430279463659191670498628785792792237461236532551241091557
- q: 12515132804905159708127766136325707992321377113998710477445811836263405233128780498152047439057240132542217905573440500557504942936575400253824837109325257

8
- p: 8649620751833675845720949489383858845796447401365353800898878144845175406172676844999275150219630606359684669242320399096050331623719946048021382644892803
- q: 11896494295276874055593281869319334151651783133808673396575485807068776793606182148494465816391713137317129560972663854384901692609500496853548961122888889

9
- p: 9578503710865082752572619447703250581238252879412822133271432204532787671643027893670256331928358458734493559747171810834463456086852684437858903431104943
- q: 10207017087152588040715048402680785398862222629910637172475793288776252558629748676497021478354292685455714591071372731030921157909006333951719151324136879

10
- p: 9686924917554805418937638872796017160525664579857640590160320300805115443578184985934338583303180178582009591634321755204008394655858254980766008932978633
- q: 9686924917554805418937638872796017160525664579857640590160320300805115443578184985934338583303180178582009591634321755204008394655858254980766008932978699

11
- p: 10567461048505039641972710268713128944634687748250712080474984695584136876402672898538333956369978635390580930966654058991315304961329166742263216335895001
- q: 10627535420870140157264943381237995845188613757283488311986534170673076876359166440468642831586000127005789867628311873382408383407839525066568105400526039

12
- p: 6985860474362742689823213380101231514167124463232248283942151780607845942841460091693138693954396501918480507964569322391710334462720533595017918609916463
- q: 12921454883136991414340641129184178393731045365222406074761149065515779165690863384311170163255060973667442222325395407973824379536104548265452893540473427

14
- p: 10954856299233465126359914171500305822846165431085183673999109759449706415739193445885099004577509868426540084786683485568001351280541116090063034118634519
- q: 10954856299233465126359914171500305822846165431085183673999109759449706417636519711881707731622506407722143163847672064459333431572992021257881551867597529

15
- p: 12129590228679741504121711843970362493049315734589299243022334854180628199056110072388206407916677887428330057500003256795368137540193144088105889375346813
- q: 12179582871512776468956891745877445010350618063676813770961929030122878089063984750168929497808121238411849832804023412100455762402728468432020965274841793

16
- p: 6998204055345503454608535735199373144581147952412423572966494183179306005216278935510553634487794945379209006773628864213102839984362642462197184843259867
- q: 12956635227542071202375560927144520333507358619831580670377595997503810379893056493870213428580918250332435506596912071784762831942734238145246920736537647

18
- p: 7273268163465293471933643674908027120929096536045429682300347130226398442391418956862476173798834057392247872274441320512158525416407044516675402521694747
- q: 12840807874760119497562989864651565491645077946976950748211992253853323703532620362223764981952516328133916264333884385029280730688894521589959051436522977

20
- p: 7006433107252813175095285828299335809650512229854304253471035955787617465825842990081320535410435973346940338772266519696972554709877157476961995216696227
- q: 12976180370825816046330723693051565324091920359016588247467647785104809172783057319885443026117787490599861213721564637212466483142611446237049740392927759


### p的长度不是16的倍数
(513,576)

2
- p: 52484065122572767557293534477361686456679280880304125291106733197354892893647364164212186415880889674435558369420400890814461263958618375991691022752189839
- q: 1719620105458406433483340568317543019584575635895742560438771105058321655238562613083979651479555788009994557822024565226932906295208262756822275663694111
  
13
- p: 28159870572597920594563893250499572739237769660647238839011417383170724985058502301163390234256825164330439886062865686161169349465086627567328776299903327
- q: 3351951982485649274893506249551461531869841455148098344430890360930441007518386744200468574541725856922507964546621512713438470702986642490397676148760429

17
- p: 33168227830849222860094691158174263663422336899723339302414624335921937096795361698659264621281924482566876836548283550012466892022559940789243695658550143
- q: 3351951982485649274893506249551461531869841455148098344430890360930441007518386744200468574541725856922511753810518751644117399610974624147782234049900131